In [3]:
import json
import string
import numpy as np

punc = string.punctuation

In [8]:
result_dic = {}
word_dic = load_word2vec(dim)
# set the dim
dim = 100
with open('/Users/zhengwei/Desktop/lesson6/社交网络挖掘/dataset/dblp-ref-0.json', 'r') as f:
    line = f.readline()
    temp_vec = np.zeros(dim)
    while line:        
        line = json.loads(line)
        if 'abstract' in line:
            abstract_ = line['abstract'].split(' ')
            length = len(abstract_)
            for item in abstract_:
                if item == '':
                    continue
                if item[0] in punc:
                    item = item[1:]
#                 if item == '':
#                     print(abstract_)
                if item == '':
                    continue
                if item[-1] in punc:
                    item = item[:-1]

                if item in word_dic:
                    temp_vec += np.array(word_dic[item], dtype=np.float64)
                else:
                    continue
            result_dic[line['id']] = [temp_vec/length]
        else:
            result_dic[line['id']] = [temp_vec]
        
        temp_vec = np.zeros(dim)
        title_ = line['title'].split(' ')
        length = len(title_)
        for item in title_:
            if item == '':
                    continue
            if item[0] in punc:
                item = item[1:]
            if item == '':
                    continue
            if item[-1] in punc:
                item = item[:-1]
                
            if item in word_dic:
                temp_vec += np.array(word_dic[item], dtype=np.float64)
            else:
                continue
        result_dic[line['id']].append(temp_vec/length)
        
        line = f.readline()

In [7]:

def load_word2vec(dim):
    word_dic = {}
    with open('/Users/zhengwei/Desktop/glove/glove.6B.'+ str(dim) + 'd.txt') as f:
        line = f.readline()
        while line:
            line = line[:-1].split(' ')
            word_dic[line[0]] = line[1:]
            line = f.readline()
    return word_dic

In [1]:
import pickle

def write_in_pickle(data, path):
    output = open(path, 'wb')
    pickle.dump(data, output)
    output.close()

def load_pickle(path):
    pkl_file = open(path, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data

In [9]:
write_in_pickle(result_dic)

In [1]:
### Task 2

import json

author_dic = {}
with open('/Users/zhengwei/Desktop/lesson6/社交网络挖掘/dataset/dblp-ref-0.json', 'r') as f:
    line = f.readline()
    while line:  
        line = json.loads(line)
        author_list = line['authors']
        for i in range(len(author_list)):
            main_author = author_list[i]
            if main_author not in author_dic:
                author_dic[main_author] = {}
            coo_author = author_list[:i] + author_list[i+1:]
            for author in coo_author:
                if author not in author_dic[main_author]:
                    author_dic[main_author][author] = 1
                else:
                    author_dic[main_author][author] += 1
        line = f.readline()

In [13]:
write_in_pickle(author_dic, '/Users/zhengwei/Desktop/coo_author.pkl')

In [3]:
data = load_pickle('/Users/zhengwei/Desktop/coo_author.pkl')

In [4]:
data['Gareth Beale']

{'Graeme Earl': 1, 'Ian Dawson': 1, 'Louisa Minkin': 1, 'Nicole Beale': 1}